In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [0]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/MinorProject/hate-speech-dataset-master.zip", 'r')
zip_ref.extractall()
zip_ref.close()
file_path="/content/hate-speech-dataset-master/all_files"

In [51]:
read_files=glob.glob(os.path.join(file_path,"*.txt"))
data=[]
file_name=[]
for files in read_files:
    with open(files,encoding='utf-8') as f:
        data.append(f.read())
        file_name.append(os.path.splitext(os.path.basename(files))[0])
data=pd.DataFrame(data,index=file_name)
data.head()

0
13639258_1  I just cannot understand someone wanting to ri...
30680004_3  What educational goals or philosophies influen...
13504039_3  Los Angeles County sheriff 's officials arrest...
13465908_1  France has been gone for a long time , we will...
14665319_1                                     84% French ...

In [52]:
Output=pd.read_csv("/content/hate-speech-dataset-master/annotations_metadata.csv",index_col=0)
Output=Output.drop(['user_id','subforum_id','num_contexts'],axis=1).sort_index()
Output.head()

label
file_id            
12834217_1   noHate
12834217_10  noHate
12834217_2   noHate
12834217_3   noHate
12834217_4     hate

In [53]:
dataSet=pd.merge(data,Output,how='outer',left_index=True,right_index=True)
dataSet=dataSet[(dataSet['label']=="noHate")|(dataSet['label']=='hate')]
dataSet.rename(columns={0:'Texts'},inplace=True)
dataSet['label']=(dataSet['label']=='hate').astype(int)
dataSet.head()

Texts  label
12834217_1   As of March 13th , 2014 , the booklet had been...      0
12834217_10  Thank you in advance. : ) Download the youtube...      0
12834217_2   In order to help increase the booklets downloa...      0
12834217_3   ( Simply copy and paste the following text int...      0
12834217_4   Click below for a FREE download of a colorfull...      1

In [54]:
x = dataSet['Texts']
y = dataSet['label']
print(x.head(), y.head())

12834217_1     As of March 13th , 2014 , the booklet had been...
12834217_10    Thank you in advance. : ) Download the youtube...
12834217_2     In order to help increase the booklets downloa...
12834217_3     ( Simply copy and paste the following text int...
12834217_4     Click below for a FREE download of a colorfull...
Name: Texts, dtype: object 12834217_1     0
12834217_10    0
12834217_2     0
12834217_3     0
12834217_4     1
Name: label, dtype: int64


In [55]:
from keras.utils.np_utils import to_categorical
y = to_categorical(np.asarray(y))
print(x)

12834217_1     As of March 13th , 2014 , the booklet had been...
12834217_10    Thank you in advance. : ) Download the youtube...
12834217_2     In order to help increase the booklets downloa...
12834217_3     ( Simply copy and paste the following text int...
12834217_4     Click below for a FREE download of a colorfull...
                                     ...                        
33677015_1     Apparently he came to the conclusion that his ...
33677019_1     Wish we at least had a Marine Le Pen to vote f...
33677019_2     Its like the choices are white genocide candid...
33677053_1     Why White people used to say that sex was a si...
33677053_2                                        Now I get it !
Name: Texts, Length: 10703, dtype: object


In [0]:
tk=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(x)
word_index=tk.word_index
#print(index)
x2 = tk.texts_to_sequences(x)

In [57]:
embedding_vecor_length =146
padded_docs = pad_sequences(x2, maxlen=embedding_vecor_length, padding='post')
print (padded_docs)

[[ 26   6 515 ...   0   0   0]
 [200  11   7 ...   0   0   0]
 [  7 701   2 ...   0   0   0]
 ...
 [111  28   1 ...   0   0   0]
 [148  18  43 ...   0   0   0]
 [100   3  53 ...   0   0   0]]


In [58]:
vocab_size = len(tk.word_index)
print(vocab_size)

16002


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.30, random_state=42)

In [0]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/MinorProject/crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [61]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [0]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [63]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(input_dim = vocab_size + 1, output_dim = 300, weights=[embedding_matrix], input_length=embedding_vecor_length))
model.add(Conv1D(filters = 128,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 146, 300)          4800900   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 144, 128)          115328    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 28, 128)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3584)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               917760    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
Total params: 5,834,502
Trainable params: 5,834,502
Non-trainable params: 0
____________________________________________

In [64]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=100, verbose=2)

Train on 7492 samples, validate on 3211 samples
Epoch 1/100
 - 2s - loss: 0.3412 - acc: 0.8877 - val_loss: 0.2547 - val_acc: 0.8938
Epoch 2/100
 - 1s - loss: 0.1947 - acc: 0.9221 - val_loss: 0.2289 - val_acc: 0.9097
Epoch 3/100
 - 1s - loss: 0.0948 - acc: 0.9652 - val_loss: 0.2514 - val_acc: 0.9081
Epoch 4/100
 - 1s - loss: 0.0300 - acc: 0.9925 - val_loss: 0.3093 - val_acc: 0.9078
Epoch 5/100
 - 1s - loss: 0.0084 - acc: 0.9985 - val_loss: 0.3716 - val_acc: 0.9056
Epoch 6/100
 - 1s - loss: 0.0032 - acc: 0.9997 - val_loss: 0.4689 - val_acc: 0.9072
Epoch 7/100
 - 1s - loss: 0.0018 - acc: 0.9999 - val_loss: 0.4194 - val_acc: 0.9063
Epoch 8/100
 - 1s - loss: 0.0012 - acc: 0.9997 - val_loss: 0.5081 - val_acc: 0.9084
Epoch 9/100
 - 1s - loss: 4.6946e-04 - acc: 1.0000 - val_loss: 0.4923 - val_acc: 0.9053
Epoch 10/100
 - 1s - loss: 2.8174e-04 - acc: 1.0000 - val_loss: 0.5102 - val_acc: 0.9066
Epoch 11/100
 - 1s - loss: 2.0848e-04 - acc: 1.0000 - val_loss: 0.5298 - val_acc: 0.9059
Epoch 12/100
 

In [65]:
predictions_test = model.predict(X_test)
print(predictions_test)

[[1.0000000e+00 4.2393983e-12]
 [1.0000000e+00 9.2529798e-14]
 [1.0000000e+00 5.5785706e-16]
 ...
 [1.0000000e+00 3.3898338e-12]
 [9.9999917e-01 7.9459977e-07]
 [1.0000000e+00 2.1026852e-12]]


In [0]:
predictions_test1 = np.zeros_like(predictions_test)
predictions_test1[np.arange(len(predictions_test)), predictions_test.argmax(1)] = 1

In [67]:
print(classification_report(y_test,predictions_test1))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95      2842
           1       0.62      0.45      0.52       369

   micro avg       0.91      0.91      0.91      3211
   macro avg       0.78      0.71      0.74      3211
weighted avg       0.90      0.91      0.90      3211
 samples avg       0.91      0.91      0.91      3211

